In [1]:
import json
import pandas as pd

# Load the JSON file
with open("dev_tables.json", "r", encoding="utf-8") as f:
    data = json.load(f)

table_schemas = {}
dbs = []

for db in data:
    db_id = db.get("db_id", [])
    dbs.append(db_id)
    schema_promt = f"Below are the descriptions for the database {db_id}:\n"
    for table in db.get("table_names_original", []):
        # import csv file in dev_databases/{db_id}/database_description/{table}.csv
        try:
            table_desc = pd.read_csv(f"dev_databases/{db_id}/database_description/{table}.csv")
        except Exception as e:
            print(db_id, table, e)
        table_index = db["table_names_original"].index(table)
        schema_promt += f"[Table name]\n{table}\n[(Column name, description, type)]\n"
        for i, (tbl_idx, col_name) in enumerate(db.get("column_names_original", [])):
            if tbl_idx == table_index:
                # find the column description whose original_column_name is col_name
                try:
                    col_desc = table_desc[table_desc["original_column_name"] == col_name]["column_description"].values[0]
                except Exception as e:
                    col_desc = ""
                type_desc = db.get("column_types", [])[i]
                schema_promt += f"{col_name}, {col_desc}, {type_desc}\n"
        schema_promt += "\n"
    table_schemas[db_id] = schema_promt

In [80]:
def next():
    results = json.load(open("eval_results_schema_desc.json", "r"))
    for db_id in results:
        schema_promt = table_schemas[db_id]
        for query in results[db_id]:
            if query["result"] == "wrong answer" or query["result"] == "not executable":
                yield schema_promt, query["question"], query["gold_query"], query["generated_query"], query["result"], query
generator = next()

In [ ]:
reason = []

In [45]:
ambiguity = "ambiguity"
wrong_function = "wrong sql function"
wrong_reasoning = "wrong reasoning"
wrong_schema = "wrong schema"
wrong_value = "wrong value"
wrong_gold = "wrong gold"

In [81]:
example = generator.__next__()
for str in example[:-1]:
    print(str)

Below are the descriptions for the database debit_card_specializing:
[Table name]
customers
[(Column name, description, type)]
CustomerID, identification of the customer, integer
Segment, client segment, text
Currency, Currency, text

[Table name]
gasstations
[(Column name, description, type)]
GasStationID, Gas Station ID, integer
ChainID, Chain ID, integer
Country, nan, text
Segment, chain segment, text

[Table name]
products
[(Column name, description, type)]
ProductID, Product ID, integer
Description, Description, text

[Table name]
transactions_1k
[(Column name, description, type)]
TransactionID, Transaction ID, integer
Date, Date, date
Time, Time, text
CustomerID, Customer ID, integer
CardID, Card ID, integer
GasStationID, Gas Station ID, integer
ProductID, Product ID, integer
Amount, Amount, integer
Price, Price, real

[Table name]
yearmonth
[(Column name, description, type)]
CustomerID, Customer ID, integer
Date, Date, text
Consumption, consumption, real


What was the average m

In [71]:
query = example[-1]
query["reason"] = wrong_value
reason.append(query)

In [74]:
nambiguity = 0
nwrong_function = 0
nwrong_reasoning = 0
nwrong_schema = 0
nwrong_value = 0
nwrong_gold = 0

for query in reason:
    if ambiguity in query["reason"]:
        nambiguity += 1
    if wrong_function in query["reason"]:
        nwrong_function += 1
    if wrong_reasoning in query["reason"]:
        nwrong_reasoning += 1
    if wrong_schema in query["reason"]:
        nwrong_schema += 1
    if wrong_value in query["reason"]:
        nwrong_value += 1
    if wrong_gold in query["reason"]:
        nwrong_gold += 1

In [78]:
reason = sorted(reason, key=lambda k:k["reason"])

In [79]:
with open("error_analysis.json", "w", encoding="utf-8") as f:
    json.dump(reason, f, indent=2)

In [75]:
print(nambiguity, nwrong_function, nwrong_reasoning, nwrong_schema, nwrong_value, nwrong_gold)

3 1 5 5 7 3


schema ambiguity
language ambiguity

What is the difference in the annual average consumption of the customers with the least amount of consumption paid in CZK for 2013 between SME and LAM, LAM and KAM, and KAM and SME?